In [2]:
import numpy as np
import pandas as pd

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# UMAP
import umap
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE

In [3]:
# Set random seed
# np.random.seed(2024)

In [4]:
# Function to generate data for Scenario A
def generate_X_a(num_samples, p):
    return np.random.normal(0, 1, (num_samples, p))

# Function to generate data for Scenario B
def generate_X_b(num_samples, p):
    X1 = np.random.normal(-1, 1, (num_samples // 2, p))
    X2 = np.random.normal(1, 1, (num_samples // 2, p))
    X = np.vstack((X1,X2))
    np.random.shuffle(X)
    return X

# Function to generate data for Scenario C
def generate_X_c(num_samples, p):
    mean = np.zeros(p)
    cov = 0.6 * np.eye(p) + 0.4 * np.ones((p, p))
    return np.random.multivariate_normal(mean, cov, num_samples)


# Function to generate responses for each model
def generate_responses(X, subset, epsilon):
    
    # Model I
    sum_squares = np.sum(X[:, :subset]**2, axis=1)
    Y1 = np.sqrt(sum_squares) * np.log(np.sqrt(sum_squares)) + epsilon
    
    # Model II
    terms = [(X[:, i] / (1 + np.exp(X[:, i + 1]))) for i in range(int(subset))]
    Y2 = np.sum(terms, axis=0) + epsilon
    
    # Model III
    sum = np.sum(X[:, :10], axis=1)
    Y3 = np.sin((np.pi * sum_squares) / 10) + epsilon
    
    return Y1, Y2, Y3
# Set the number of subsets related to responses
#Generate a transforming function
def generate_df(X,Y1,Y2,Y3):
    df = pd.DataFrame(X, columns = columns)
    df['Y1'] = Y1
    df['Y2'] = Y2
    df['Y3'] = Y3
    return df 

### Sliced Inversed Regression
import statsmodels.api as sm

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

class SlicedInverseRegression(BaseEstimator, TransformerMixin):
    def __init__(self, n_directions=2, n_slices=10):
        self.n_directions = n_directions
        self.n_slices = n_slices

    def fit(self, X, y):
        n_samples = X.shape[0]
        self.X_mean_ = np.mean(X, axis=0)

        # Center X
        X_centered = X - self.X_mean_

        # Slice y and compute the slice means of X
        sorted_indices = np.argsort(y)
        slice_size = n_samples // self.n_slices

        M = np.zeros((X.shape[1], X.shape[1]))
        for i in range(self.n_slices):
            slice_indices = sorted_indices[i * slice_size:(i + 1) * slice_size]
            X_slice = X_centered[slice_indices]
            slice_mean = np.mean(X_slice, axis=0)
            M += slice_size * np.outer(slice_mean, slice_mean)

        # Eigen decomposition
        eigvals, eigvecs = np.linalg.eigh(M)
        self.directions_ = eigvecs[:, -self.n_directions:]

        return self

    def transform(self, X):
        check_is_fitted(self, 'directions_')
        return np.dot(X - self.X_mean_, self.directions_)

### Kernel Sliced Inversed Regression
import numpy as np
from scipy.linalg import eigh
from sklearn.metrics.pairwise import rbf_kernel

class KernelSIR:
    def __init__(self, num_components=2, n_slices=10, gamma=0.1):
        self.num_components = num_components
        self.n_slices = n_slices
        self.gamma = gamma
        self.W = None

    def fit(self, X, y):
        # Compute the kernel matrix
        K = self.compute_kernel(X)
        
        # Slice the data based on the response variable y
        slices = self.slice_data(y)
        
        # Compute the kernel SIR matrix
        M = self.kernel_sir_matrix(K, slices)
        
        # Eigen decomposition to get the projection matrix W
        self.W = self.eig_decomposition(M)
        
    def transform(self, X):
        if self.W is None:
            raise ValueError("The model has not been fitted yet. Call 'fit' with appropriate arguments before using this method.")
        # Compute the kernel matrix for the new data
        K = self.compute_kernel(X)
        # Transform the data using the projection matrix W
        return np.dot(K, self.W)
    
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X)
    
    def compute_kernel(self, X):
        return rbf_kernel(X, gamma=self.gamma)

    def slice_data(self, y):
        y = np.asarray(y)
        n = len(y)
        sorted_indices = np.argsort(y)
        slices = np.array_split(sorted_indices, self.n_slices)
        return slices

    def kernel_sir_matrix(self, K, slices):
        n = K.shape[0]
        N_h = np.zeros(len(slices))
        M_h = np.zeros((n, n))
        
        for h, indices in enumerate(slices):
            N_h[h] = len(indices)
            K_h = K[indices, :]
            K_h_mean = np.mean(K_h, axis=0)
            M_h += (N_h[h] / n) * np.outer(K_h_mean, K_h_mean)
        
        K_mean = np.mean(K, axis=0)
        M = M_h - np.outer(K_mean, K_mean)
        
        return M

    def eig_decomposition(self, M):
        eigvals, eigvecs = eigh(M)
        sorted_indices = np.argsort(eigvals)[::-1]
        top_eigvecs = eigvecs[:, sorted_indices[:self.num_components]]
        return top_eigvecs

#Generate a fuction to calculate the knn score
def knnmse(X_train,y_train,X_test,y_test):
    
    knn_regressor = KNeighborsRegressor(n_neighbors=3)
    knn_regressor.fit(X_train, y_train)
    
    y_pred_train = knn_regressor.predict(X_train)
    mse_train = round(mean_squared_error(y_train, y_pred_train),3)

    y_pred_test = knn_regressor.predict(X_test)
    mse_test = round(mean_squared_error(y_test, y_pred_test),3)
    
    return [mse_train, mse_test]

In [5]:
# Number of samples, dimensions, and relative features to Responses
num_samples = 1000
p = 500
subset = 10
# Noise term
epsilon = np.random.normal(0, np.sqrt(0.25), num_samples)
# Transform np.array into dataframe
columns= [f'feature{i+1}' for i in range(p)]

In [6]:
def run_models():
    ### Data simulation
    # Generate data for each scenario
    X_a = generate_X_a(num_samples, p)
    X_b = generate_X_b(num_samples, p)
    X_c = generate_X_c(num_samples, p)
     
    # Generate responses for Xs in each scenario
    Y_a1, Y_a2, Y_a3 = generate_responses(X_a,subset, epsilon)
    Y_b1, Y_b2, Y_b3 = generate_responses(X_b,subset, epsilon)
    Y_c1, Y_c2, Y_c3 = generate_responses(X_c, subset,epsilon)
    
    # Combine the data
    data_a = np.column_stack((X_a, Y_a1, Y_a2, Y_a3))
    data_b = np.column_stack((X_b, Y_b1, Y_b2, Y_b3))
    data_c = np.column_stack((X_c, Y_c1, Y_c2, Y_c3))
    
    
    # Transform array Xs with three of their responses into one dataframe.
    df_a = generate_df(X_a,Y_a1, Y_a2, Y_a3)
    df_b = generate_df(X_b,Y_b1, Y_b2, Y_b3)
    df_c = generate_df(X_c,Y_c1, Y_c2, Y_c3)
    
    dfs = [df_a,df_b,df_c]
    
    # Proprocess the dataset, and cut the continuous response into 10 slices. Separate
    # these datasets into training and testing parts.
    
    # Initialize lists of functional groups to keep separated parts of all the datasets
    Ys =['Y1','Y2','Y3']
    
    Xs=[]
    
    X_trains = []
    
    X_tests = []
    
    y_sli_trains = []
    
    y_sli_tests = []
    
    y_orig_trains = []
    
    y_orig_tests = []
    
    y_mean_trains = []
    
    y_mean_tests = []
    
    y_means = []
    
    for df in dfs:
        X = df.drop(['Y1','Y2','Y3'],axis=1)
        Xs.append(X)
        for response in Ys:
            ## The first four lines cut responses into 5 slices for further experiment.
            df[f'{response}_sli'] = pd.qcut(df[response], q=5, labels=False)
            y = df[f'{response}_sli'].values.flatten()
            X_train, X_test, y_sli_train, y_sli_test, train_indices, test_indices =\
            train_test_split(X, y, df.index, test_size=0.5)
            ## The end of the slices cut.
            y_orig_train = df[response].iloc[train_indices]
            y_orig_test = df[response].iloc[test_indices]
            y_mean_train = round(df[response].iloc[train_indices].mean(),3)
            y_mean_test = round(df[response].iloc[test_indices].mean(),3)
            y_mean = round(df[response].mean(),3)
            
            X_trains.append(X_train)
            X_tests.append(X_test)
            y_sli_trains.append(y_sli_train)
            y_sli_tests.append(y_sli_test)
            y_orig_trains.append(y_orig_train)
            y_orig_tests.append(y_orig_test)
            y_mean_trains.append(y_mean_train)
            y_mean_tests.append(y_mean_test)
            y_means.append(y_mean)
    
    ## Supervised UMAP model with sliced responses
    
    # Initialize supervised UMAP
    n_neighbors = 5
    umap_mod = umap.UMAP(n_neighbors=n_neighbors)
    
    # Fit and Transform three dataset
    train_ems = []
    test_ems = []
    for i in range(len(X_trains)):
        umap_train = umap_mod.fit_transform(X_trains[i],y_sli_trains[i])
        umap_test = umap_mod.transform(X_tests[i])
        
        train_ems.append(umap_train)
        test_ems.append(umap_test)
    

    
    # Initialize lists to store the mse s
    org_mses = []
    umap_sup_mses = []
    test_mse = []
    train_mse = []
    
    # The first two groups stored in the lists are ori and sup umap, each group has nine items for nine datasets
    # Loop through the data and calculate the mse s
    for i in range(len(X_trains)):
        org_mse = knnmse(X_trains[i], y_orig_trains[i], X_tests[i], y_orig_tests[i])
        
        org_mses.append(org_mse)
        test_mse.append(org_mse[1])
        train_mse.append(org_mse[0])

  
    
    for i in range(len(X_trains)):
        umap_sup_mse = knnmse(train_ems[i], y_orig_trains[i], test_ems[i], y_orig_tests[i])
        
        umap_sup_mses.append(umap_sup_mse)
        test_mse.append(umap_sup_mse[1])
        train_mse.append(umap_sup_mse[0])
        
    
    ### Supervised UMAP treating response as a categorical variable by default.
    
    conti_train_ems = []
    conti_test_ems = []
    for i in range(len(X_trains)):
        umap_train = umap_mod.fit_transform(X_trains[i],y_orig_trains[i])
        umap_test = umap_mod.transform(X_tests[i])
        
        conti_train_ems.append(umap_train)
        conti_test_ems.append(umap_test)
    
    # Initialize lists to store the scores
    umap_sup_cont_mses = []
    
    # Loop through the data and calculate the scores
    for i in range(len(X_trains)):
        umap_sup_cont_mse = knnmse(conti_train_ems[i], y_orig_trains[i], conti_test_ems[i], y_orig_tests[i])
        
        umap_sup_cont_mses.append(umap_sup_cont_mse)
        
        test_mse.append(umap_sup_cont_mse[1])
        train_mse.append(umap_sup_cont_mse[0])
    
    
    
    ### Continuous response with targer_metrics setting                             
    




    conti_train_ems1 = []
    conti_test_ems1 = []
    umap_mod1 = umap.UMAP(n_neighbors=n_neighbors,target_metric='l2')
    for i in range(len(X_trains)):
        umap_train = umap_mod1.fit_transform(X_trains[i],y_orig_trains[i])
        umap_test = umap_mod1.transform(X_tests[i])
        
        conti_train_ems1.append(umap_train)
        conti_test_ems1.append(umap_test)
    
    # Initialize lists to store the scores
    umap_sup_cont_mses1 = []
    
    # Loop through the data and calculate the scores
    for i in range(len(X_trains)):
        umap_sup_cont_mse1 = knnmse(conti_train_ems1[i], y_orig_trains[i], conti_test_ems1[i], y_orig_tests[i])
        
        umap_sup_cont_mses1.append(umap_sup_cont_mse1)
        test_mse.append(umap_sup_cont_mse1[1])
        train_mse.append(umap_sup_cont_mse1[0])
    
        
    
    ### Unsupervise UMAP Model
    
    # Perform Unsupervised UMAP
    # Fit and Transform 
    un_train_ems = []
    un_test_ems = []
    for i in range(len(X_trains)):
        umap_train = umap_mod.fit_transform(X_trains[i])
        umap_test = umap_mod.transform(X_tests[i])
        
        un_train_ems.append(umap_train)
        un_test_ems.append(umap_test)
    
    # Initialize lists to store the scores
    umap_unsup_mses = []
    
    # Loop through the data and calculate the scores
    for i in range(len(X_trains)):
        umap_unsup_mse = knnmse(un_train_ems[i], y_orig_trains[i], un_test_ems[i], y_orig_tests[i])
        
        umap_unsup_mses.append(umap_unsup_mse)
        test_mse.append(umap_unsup_mse[1])
        train_mse.append(umap_unsup_mse[0])
    
    ### PCA
    
    # Initialize PCA
    pca = PCA(n_components=2)
    
    # Fit and Transform three dataset
    pca_trains = []
    pca_tests = []
    for i in range(len(X_trains)):
        pca_train = pca.fit_transform(X_trains[i])
        pca_test = pca.transform(X_tests[i])
        
        pca_trains.append(pca_train)
        pca_tests.append(pca_test)
    
    # Loop through the data and calculate the KNN scores
    pca_mses = []
    for i in range(len(X_trains)):
        pca_mse = knnmse(pca_trains[i], y_orig_trains[i], pca_tests[i], y_orig_tests[i])
        
        pca_mses.append(pca_mse)
        test_mse.append(pca_mse[1])
        train_mse.append(pca_mse[0])
        
    
    
    ### KernelPCA
    
    # Initialize kernel PCA
    kpca = KernelPCA(n_components=2,kernel='rbf')
    
    # Fit and Transform three dataset
    kpca_trains = []
    kpca_tests = []
    for i in range(len(X_trains)):
        kpca_train = kpca.fit_transform(X_trains[i])
        kpca_test = kpca.transform(X_tests[i])
        
        kpca_trains.append(kpca_train)
        kpca_tests.append(kpca_test)
    
    # Loop through the data and calculate the KNN scores
    kpca_mses = []
    for i in range(len(X_trains)):
        kpca_mse = knnmse(kpca_trains[i], y_orig_trains[i], kpca_tests[i], y_orig_tests[i])
        
        kpca_mses.append(kpca_mse)
        test_mse.append(kpca_mse[1])
        train_mse.append(kpca_mse[0])
    
    ### Sliced Inversed Regression
    
    # Initialize SIR
    sir = SlicedInverseRegression(n_directions=2, n_slices=10)
    
    # Fit and Transform three dataset
    sir_trains = []
    sir_tests = []
    for i in range(len(X_trains)):
        sir_train = sir.fit_transform(np.array(X_trains[i]),y_orig_trains[i])
        sir_test = sir.transform(np.array(X_tests[i]))
        
        sir_trains.append(sir_train)
        sir_tests.append(sir_test)
    
    # Loop through the data and calculate the KNN scores
    sir_mses = []
    for i in range(len(X_trains)):
        sir_mse = knnmse(sir_trains[i], y_orig_trains[i], sir_tests[i], y_orig_tests[i])
        
        sir_mses.append(sir_mse)
        test_mse.append(sir_mse[1])
        train_mse.append(sir_mse[0])
    
    ### Kernel Sliced Inversed Regression
    # Initialize kernerl SIR
    ksir = KernelSIR(num_components=2, n_slices=10, gamma=0.1)
    
    # Fit and Transform three dataset
    ksir_trains = []
    ksir_tests = []
    for i in range(len(X_trains)):
        ksir_train = ksir.fit_transform(np.array(X_trains[i]),y_orig_trains[i])
        ksir_test = ksir.transform(np.array(X_tests[i]))
        
        ksir_trains.append(ksir_train)
        ksir_tests.append(ksir_test)
    
    # Loop through the data and calculate the KNN scores
    ksir_mses = []
    for i in range(len(X_trains)):
        ksir_mse = knnmse(ksir_trains[i], y_orig_trains[i], ksir_tests[i], y_orig_tests[i])
        
        ksir_mses.append(ksir_mse)
        test_mse.append(ksir_mse[1])
        train_mse.append(ksir_mse[0])
    
    ### t-Distributed Stochastic Neighbor Embedding (t-SNE)
    
    # Initialize tsne
    tsne = TSNE(n_components=2)
    ## TNSE doesn't have a seperate transform function, we use fit_transform for testing data.
    ## Therefore the testing dataset fits its own model instead of useing the trainning model.
    
    # Fit and Transform three dataset
    tsne_trains = []
    tsne_tests = []
    for i in range(len(X_trains)):
        tsne_train = tsne.fit_transform(X_trains[i])
        tsne_test = tsne.fit_transform(X_tests[i])
        
        tsne_trains.append(tsne_train)
        tsne_tests.append(tsne_test)
    
    # Loop through the data and calculate the KNN scores
    tsne_mses = []
    for i in range(len(X_trains)):
        tsne_mse = knnmse(tsne_trains[i], y_orig_trains[i], tsne_tests[i], y_orig_tests[i])
        
        tsne_mses.append(tsne_mse)
        test_mse.append(tsne_mse[1])
        train_mse.append(tsne_mse[0])
    # print('Train', train_mse)
    # print('Test', test_mse)
    return train_mse, test_mse
    

In [12]:
# Set a timer
import time
train_mses = []
test_mses = []
start_time = time.time()
for i in range(100):
    train_mse,test_mse = run_models()
    train_mses.append(train_mse)
    test_mses.append(test_mse)
    print(f'The {i+1}th run')
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed_time',elapsed_time)

/Users/guanzhezhang/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


The 1th run


/Users/guanzhezhang/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


The 2th run
The 3th run
The 4th run
The 5th run


/Users/guanzhezhang/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


...

The 90th run
The 91th run
The 92th run
The 93th run


/Users/guanzhezhang/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


The 94th run
The 95th run
The 96th run


/Users/guanzhezhang/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


The 97th run
The 98th run
The 99th run
The 100th run
Elapsed_time 9054.480657815933


In [13]:
np.savetxt('output/0105/100-time array test_mses 0105.csv', test_mses, delimiter=',', fmt='%.4f')
np.savetxt('output/0105/100-time array train_mses 0105.csv', train_mses, delimiter=',', fmt='%.4f')

In [14]:
train_mses = np.array(train_mses)
test_mses = np.array(test_mses)

Average_train_mses = train_mses.mean(axis=0)
Average_test_mses = test_mses.mean(axis=0)

n_rows = test_mses.shape[0]
std_dev_test = np.std(test_mses, axis=0, ddof=1)
std_err_test = std_dev_test / np.sqrt(n_rows)

Table_test_mses = Average_test_mses.reshape(10,9).T
Table_train_mses = Average_train_mses.reshape(10,9).T
Table_test_se = std_err_test.reshape(10,9).T


### Create a table of scores

In [16]:
std_err_test

array([0.0261733 , 0.02697138, 0.00692483, 0.04273269, 0.05180093,
       0.0055587 , 0.01997627, 0.02319174, 0.00541449, 0.02974962,
       0.03479237, 0.00702568, 0.05191154, 0.07683292, 0.00687216,
       0.02515231, 0.03102818, 0.00646066, 0.02704758, 0.03441066,
       0.00785357, 0.04802267, 0.06383413, 0.00605892, 0.02007942,
       0.02817761, 0.00515799, 0.0818054 , 0.10858463, 0.02095495,
       0.14573149, 0.74675597, 0.02430234, 0.12937571, 0.2514307 ,
       0.01883907, 0.02493829, 0.03146327, 0.0063416 , 0.04469334,
       0.25515642, 0.00721431, 0.06654875, 0.16563229, 0.01883907,
       0.02343938, 0.03267823, 0.00650063, 0.04223524, 0.05489774,
       0.00535898, 0.01916085, 0.02630732, 0.00490315, 0.02627244,
       0.03419208, 0.00680703, 0.04688133, 0.06121374, 0.00589365,
       0.02003472, 0.02861412, 0.00445448, 0.02605012, 0.02010322,
       0.00738973, 0.04608339, 0.04382762, 0.00563211, 0.02038281,
       0.02660563, 0.00567356, 0.0337465 , 0.03884923, 0.00776

In [25]:
# Define the columns with MultiIndex
columns = ['Orig',
           'Sli_UMAP_Sup',
           'Cate_UMAP_Sup',
           'cont_UMAP_Sup',
           'Unsup_UMAP',
           'PCA',
           'KPCA',
           'SIR',
           'KSIR',
           't-SNE'
          ]

# Make an index
index = ['Data_a1','Data_a2','Data_a3',
        'Data_b1','Data_b2','Data_b3',
        'Data_c1','Data_c2','Data_c3'
        ]
# Create the DataFrame with the specified MultiIndex columns
TestTable = pd.DataFrame(Table_test_mses, columns=columns,index=index).round(4).add_prefix('MSE_')
Test_SE_Table = pd.DataFrame(Table_test_se, columns=columns,index=index).round(4).add_prefix('SE_')
TrainTable = pd.DataFrame(Table_train_mses, columns=columns,index=index).round(4)

combined_col = []
for col_mse, col_se in zip(TestTable.columns, Test_SE_Table.columns):
    combined_col.extend([col_mse, col_se])

TestCombined = pd.concat([TestTable, Test_SE_Table],axis=1)[combined_col]

In [27]:
TrainTable.to_csv('output/0105/100Aver_train0105.csv', index=True)

TestTable.to_csv('output/0105/100Aver_test0105.csv', index=True)

TestCombined.to_csv('output/0105/100TestCombinedSE0105.csv', index=True)

Test_SE_Table.to_csv('output/0105/100TestSE0105.csv', index=True)

In [25]:
test_mses.shape

(100, 90)